<a href="https://colab.research.google.com/github/pmontman/pub-choicemodels/blob/main/nb/solution/solu_tuto_05_ordered_logit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solution of Tutorial 5: Ordered Logit

We will see an example on how to use biogeme to estimate ordered logit models.

We will use the (in)famous 'affairs' dataset, described below.

The data comes from a survey from *Psychology Today*, conducted in the year 1969. It describes 'extramarital affairs', specifically, the participants in 
the survey were asked this question:

How often engaged in extramarital sexual intercourse during the past year?

With the possible answers:

* None
* Once
* Twice
* Three times
* 4 to 10 times
* Monthly
* Weekly
* Daily

Then by measuring socioeconomic variables of the participants, we could try to model the utility that they receive from having a extramarital affair (or more than one).

We can see that this falls perfectly into the ordered logit, the alternatives have a clear notion of order, from less affairs per year to more affairs per year, yet they are not perfectly numerical (which would allow us to use standard regression techniques).


# Description of the dataset

A scientific paper making use of the dataset to develop a 'Theory of Extramarial Affairs' can be found [here](https://www.uibk.ac.at/econometrics/data/fair78.pdf).

The dataset has 601 observations of the following variables:

* **affairs:** The answer to the survey, the answers are encoded with numbers.
  0 = None, 1 = Once, 2 = Twice, 3=Three times, 7= 4 to 10 times, 12= monthly, 12 = weekly, 12 = daily.
  As we see, the information was encoded in such a way that we lose information about the more frequent answers, and the encoded numbers do not completely coincide with the frequencies. However, there is an ordinal relationship among the possible answer, from less frequent to more frequent.

*  **gender:** Categorical variable indicating either male or female among the participants.

* **age:** Numeric variable coding age in years: 17.5 = under 20, 22 = 20–24, 27 = 25–29, 32 = 30–34, 37 = 35–39, 42 = 40–44, 47 = 45–49, 52 = 50–54, 57 = 55 or over. 

* **yearsmarried:** Numeric variable coding number of years married: 0.125 = 3 months or less, 0.417 = 4–6 months, 0.75 = 6 months–1 year, 1.5 = 1–2 years, 4 = 3–5 years, 7 = 6–8 years, 10 = 9–11 years, 15 = 12 or more years.

* **children:** Categorical variable indicating if there are children in the marriage.

* **religiousness:** Categorical variable indicating how religious in the person, encoded as numbers: 1 = anti, 2 = not at all, 3 = slightly, 4 = somewhat, 5 = very.

* **education:**: Categorical variable indicating the level of education. Encoded as numbers: 9 = grade school, 12 = high school graduate, 14 = some college, 16 = college graduate, 17 = some graduate work, 18 = master's degree, 20 = Ph.D., M.D., or other advanced degree.

* **occupation:** Categorical variable classifying the profession of the individual. Encoded as numbers, and the meaning of the numbers has been somewhat lost in time. But it could be something like the one in this [link.](https://dictionary.fitbir.nih.gov/portal/publicData/dataElementAction!view.action?dataElementName=HollingsheadJobClassCat&publicArea=true)

* **rating:** Categorical variable indicating how happy they are with the marriage. Encoded as numbers: 1 = very unhappy, 2 = somewhat unhappy, 3 = average, 4 = happier than average, 5 = very happy.

---
---

# Preparing the environment
*The preparation and dataset loading code is given to the students*

In [ ]:
!pip install biogeme

Load the packages, feel free to change the names.

In [ ]:
import pandas  as pd
import numpy as np
import matplotlib.pyplot as plt

import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.expressions as exp
import biogeme.tools as tools
import biogeme.distributions as dist

# Load the dataset

In [ ]:
path = 'https://raw.githubusercontent.com/pmontman/pub-choicemodels/main/data/affairs.csv'
affairs_pd = pd.read_csv(path)

A simple look at the dataset.

In [ ]:
affairs_pd.head(5)

,affairs,gender,age,yearsmarried,children,religiousness,education,occupation,rating
0,0,male,37.0,10.00,no,3,18,7,4
1,0,female,27.0,4.00,no,4,14,6,4
2,0,female,32.0,15.00,yes,1,12,1,4
3,0,male,57.0,15.00,yes,5,18,6,5
4,0,male,22.0,0.75,no,2,17,6,3


# Auxiliary function

In [ ]:
def qbus_update_globals_bgm(pd_df):
   globals().update(db.Database('tmp_bg_bgm_for_glob', pd_df).variables)

# Data cleaning: Preparing the dataset for Biogeme

Biogeme does not accept non-numerical variables.


We need to transform categorical variables that are not encoded into numbers.
The pandas function `factorize()` encodes a variable into integers. In this case, we will get 0 for Male, 1 for Female for the gender variable. We will get 0 for no, 1 for yes in the children varible. This means that we can use the encoding already as dummy variables that can be interpreted as 'isFemale' and 'hasChildren'.

In [ ]:
affairs_pd['gender'] = affairs_pd['gender'].factorize()[0]
affairs_pd['children'] = affairs_pd['children'].factorize()[0]


After this transformation, we can already pass the pandas dataframe to biogeme, though be mindful that some variables such as occupation are numeric but should be transformed to dummy. 

# Ordinal logit with biogeme

We will first estimate the ordered logit 'manually' using biogeme, to get a better view of the process and how it is connected to the theory. Later we can create some auxiliary functions to simplify the process.

We start by update the globals as usual.

In [ ]:
qbus_update_globals_bgm(affairs_pd)

### Utility function

The first step is to create the utility function, an important difference with respect to what we have done in multinomial logit(MNL) and nested logit is that there will be only one utility function, while in the MNL there is one utility function per alternative.

As for the variables, after some consideration (maybe you disagree), we can take
the variables religiousness, rating and education as numeric, even though they are categorical. This is because religiousness can be represented as 'intensity' of the belief in religion and education can be roughly interpreted as 'years spent in education'.

In [1]:
# Parameters to be estimated
B_gender = exp.Beta('B_gender', 0, None, None, 0)
B_age = exp.Beta('B_age', 0, None, None, 0)
B_yearsmarried = exp.Beta('B_yearsmarried', 0, None, None, 0)
B_children = exp.Beta('B_children', 0, None, None, 0)
B_religiousness = exp.Beta('B_religiousness', 0, None, None, 0)
B_education = exp.Beta('B_education', 0, None, None, 0)
B_rating = exp.Beta('B_rating', 0, None, None, 0)

NameError: ignored

Define the one utility function using our familiar syntax.
We will simplify and not apply any transformation or interaction, though it might very well improve the fitting.

In [ ]:
V_one = B_gender*gender + B_age*age + B_yearsmarried*yearsmarried + B_children*children + B_religiousness*religiousness + B_education*education + B_rating*rating

### Cutoff points

An important part of the ordinal logit model are the cut-off points, the values that will split the utility into different categories. These are being estimated from the data. 
Remember that we have to impose some order in the cutoff points, this is, tau1
is smaller than tau2, tau2 is smaller than tau3 and so on.

The way to do this in biogeme (and other estimation software) is to create some 'auxiliary' parameters that can be interpreted as the deltas or differences between taus. These deltas are created with `exp.Beta` as the any other parameter of the model.
 We impose that the deltas can be estimated, but they have to be greater than 0. Remember that when we define the parameters in Biogeme, one of the arguments of the function `exp.Beta` are the bounds that restrict the range to which the values can be estimated.

We can 'recreate' the taus from the deltas, for example, tau2 = tau1 + delta2, tau3 = tau2 + delta3 (or alternatively, tau3 = tau1 + delta2 + delta3).

Just to clarify: what we are doing with the deltas is imposing the order in the taus, it is just a trick to impose: all taus are in increasing order, starting from tau1, 'tau1' can have any value, tau2 has to be between tau1 and tau3, tau3 has to be between tau2 and tau4, ..., tau6 has to be between tau5 and +infinity. What we are doing is just a way to pass the information to biogeme.

In the next cell we declare the taus and the deltas. Notice the third argument in the `exp.Beta` when declaring the deltas is set to 0, the third argument specifies the lower bound in the estimation range. Notice how the taus are defined as the 'taus before them' plus some delta. With the exception of tau1. 
The default value of tau1 is -1, but this is arbitrary.

There next cell will be very verbose, because we have to manually define each of the deltas and taus. This is a candidate to be done automatically in an auxiliary function...

In [ ]:
tau1 = exp.Beta('tau1', -1, 0, None, 0)

delta2 = exp.Beta('delta2', 1, 0, None, 0)
tau2 = tau1 + delta2

delta3 = exp.Beta('delta3', 2, 0, None, 0)
tau3 = tau2 + delta3

delta4 = exp.Beta('delta4', 3, 0, None, 0)
tau4 = tau3 + delta4

delta5 = exp.Beta('delta5', 4, 0, None, 0)
tau5 = tau4 + delta5

What is left to define is the 'model' in the biogeme, the equivalent of the 
biogeme `models.loglogit` for the multinomial logit. In this case, we have to do it manually (another candidate for auxiliary functions).

We define the 'model' in biogeme using a dictionary. The dictionary maps the **values for the choice variable** to the computed choice probabilities. The choice variable here is affairs.

In the next cell the function `dist.logisticcdf` is just the name of the logistic transform in biogeme (there is the logistic probability distribution, which the cumulative distribution function 'logisticcdf' being the equivalent logistic transform).

In [ ]:
alt_probs_map = {
    0: dist.logisticcdf(tau1 - V_one),
    1: dist.logisticcdf(tau2 - V_one) - dist.logisticcdf( tau1 - V_one),
    2: dist.logisticcdf(tau3 - V_one) - dist.logisticcdf( tau2 - V_one),
    3: dist.logisticcdf(tau4 - V_one) - dist.logisticcdf( tau3 - V_one),
    7: dist.logisticcdf(tau5 - V_one) - dist.logisticcdf( tau4 - V_one),
    12: 1- dist.logisticcdf(tau5 - V_one)}

Then we take the log of the choice probabilities (for the loglikelihood) and specify which variable in the dataset contains the choice or alternatives.

In [ ]:

logprob = exp.log(exp.Elem(alt_probs_map, affairs))

We declare the biogeme object as usual, and we estimate the model.

In [ ]:
affairs_db = db.Database('affairs', affairs_pd)

biogeme  = bio.BIOGEME(affairs_db, logprob)

results = biogeme.estimate()

And here are the results, for your interpretation!
The new part are the taus and deltas. We have tau1, and then the deltas that will allow us to recover the remaining taus. We check tha everything looks OK,all deltas are positive.

In [ ]:
results.getEstimatedParameters()

,Value,Active bound,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
B_age,-0.034204,0.0,0.017319,-1.974883,4.828140e-02,0.017692,-1.933296,5.319980e-02
B_children,0.379980,0.0,0.291037,1.305605,1.916867e-01,0.317226,1.197819,2.309874e-01
B_education,0.095784,0.0,0.045581,2.101396,3.560620e-02,0.049239,1.945286,5.174061e-02
B_gender,-0.072024,0.0,0.218593,-0.329489,7.417863e-01,0.224468,-0.320865,7.483126e-01
B_rating,-0.454262,0.0,0.087616,-5.184684,2.163809e-07,0.090662,-5.010514,5.428495e-07
B_religiousness,-0.311164,0.0,0.087459,-3.557826,3.739367e-04,0.090100,-3.453538,5.532846e-04
B_yearsmarried,0.101451,0.0,0.031527,3.217908,1.291293e-03,0.032843,3.088919,2.008864e-03
delta2,0.368619,0.0,0.061462,5.997529,2.003427e-09,0.059607,6.184160,6.243421e-10
delta3,0.217144,0.0,0.051803,4.191768,2.767891e-05,0.052117,4.166478,3.093422e-05
delta4,0.276375,0.0,0.062230,4.441172,8.947010e-06,0.062077,4.452171,8.500662e-06


Simulation is as usual, pass the biogeme database and the dictionary with the targets to compute. In this case, to get the choice probabilities, we can reuse the `alt_probs_map` dictionary as the targets. 


In [ ]:
  bgm_pred_model = bio.BIOGEME(affairs_db, alt_probs_map)
  simulatedValues = bgm_pred_model.simulate(results.getBetaValues())
  simulatedValues

,0,1,2,3,7,12
0,0.782017,0.056343,0.027312,0.029021,0.059054,0.046253
1,0.909739,0.026041,0.011879,0.012131,0.023241,0.016969
2,0.560603,0.087846,0.047773,0.055113,0.128662,0.120002
3,0.895752,0.029746,0.013657,0.014004,0.026994,0.019846
4,0.737532,0.064937,0.032180,0.034711,0.072334,0.058306
...,...,...,...,...,...,...
596,0.839349,0.043740,0.020622,0.021512,0.042550,0.032227
597,0.619476,0.082337,0.043371,0.048858,0.109405,0.096553
598,0.724191,0.067305,0.033578,0.036388,0.076395,0.062144
599,0.445283,0.091866,0.053347,0.064798,0.166574,0.178132


To simulate the utilities we have to create a dictionary just with the utility funcion. 

In [ ]:
V_pred_map = {
    'util': V_one }

We do the simulation as usual, notice how the more positive the utility, the more it tends towars more infidelities. This is because we considered arbitrarily to intepret the 'utility that is received from having an affair'. We could have also used the 'utility from being faithful'.

In [ ]:
  bgm_pred_util = bio.BIOGEME(affairs_db, V_pred_map)
  simulated_util = bgm_pred_util.simulate(results.getBetaValues())
  simulated_util

,util
0,-1.277462
1,-2.310451
2,-0.243611
3,-2.150894
4,-1.033181
...,...
596,-1.653394
597,-0.487324
598,-0.965346
599,0.219747


# Exercise 1) Create a more fine tuned ordered logit model


We will create nonlinearities in years married, to capture the possibility that the effects are non-monotonic, for example, utility from being unfaithful increases with the years of marriage (maybe for the first few decades), but decays after being married for a very long time.

So we create a pandas dataframe with the transformed variables.

In [ ]:
affairs_nl = affairs_pd.copy()
affairs_nl['yearsmarried_sq'] = affairs_nl['yearsmarried']**2
affairs_nl['age_sq'] = affairs_nl['age']**2

The betas.

In [ ]:
# Parameters to be estimated
B_gender_nl = exp.Beta('B_gender_nl', 0, None, None, 0)
B_age_nl = exp.Beta('B_age_nl', 0, None, None, 0)
B_age_sq_nl = exp.Beta('B_age_sq_nl', 0, None, None, 0)
B_yearsmarried_nl = exp.Beta('B_yearsmarried_nl', 0, None, None, 0)
B_yearsmarried_sq_nl = exp.Beta('B_yearsmarried_sq_nl', 0, None, None, 0)
B_children_nl = exp.Beta('B_children_nl', 0, None, None, 0)
B_religiousness_nl = exp.Beta('B_religiousness_nl', 0, None, None, 0)
B_education_nl = exp.Beta('B_education_nl', 0, None, None, 0)
B_rating_nl = exp.Beta('B_rating_nl', 0, None, None, 0)


In [ ]:
qbus_update_globals_bgm(affairs_nl)

We declare the utility function.

In [ ]:
V_nl = B_gender_nl*gender + B_age_nl*age + B_age_sq_nl + B_yearsmarried_nl*yearsmarried + \
 B_yearsmarried_sq_nl*yearsmarried_sq + B_children_nl*children + \
  B_religiousness_nl*religiousness + B_education_nl*education + B_rating_nl*rating

We declare the taus, note that we this is not necessary, we could reuse the expressions, but we do this for clarity.

In [ ]:
tau1_nl = exp.Beta('tau1_nl', -1, 0, None, 0)

delta2_nl = exp.Beta('delta2_nl', 1, 0, None, 0)
tau2_nl = tau1_nl + delta2_nl

delta3_nl = exp.Beta('delta3_nl', 2, 0, None, 0)
tau3_nl = tau2_nl + delta3_nl

delta4_nl = exp.Beta('delta4_nl', 3, 0, None, 0)
tau4_nl = tau3_nl + delta4_nl

delta5_nl = exp.Beta('delta5_nl', 4, 0, None, 0)
tau5_nl = tau4_nl + delta5_nl

The dictionary that maps the alternatives to the choice probabilities.

In [ ]:
alt_probs_map_nl = {
    0: dist.logisticcdf(tau1_nl - V_nl),
    1: dist.logisticcdf(tau2_nl - V_nl) - dist.logisticcdf( tau1_nl - V_nl),
    2: dist.logisticcdf(tau3_nl - V_nl) - dist.logisticcdf( tau2_nl - V_nl),
    3: dist.logisticcdf(tau4_nl - V_nl) - dist.logisticcdf( tau3_nl - V_nl),
    7: dist.logisticcdf(tau5_nl - V_nl) - dist.logisticcdf( tau4_nl - V_nl),
    12: 1- dist.logisticcdf(tau5_nl - V_nl)}

The model

In [ ]:
logprob_nl = exp.log(exp.Elem(alt_probs_map_nl, affairs))

Finally the biogeme object and we estimate the model.

In [ ]:
affairs_nl_db = db.Database('affairs_nl', affairs_nl)

biogeme_nl  = bio.BIOGEME(affairs_nl_db, logprob_nl)

results_nl = biogeme_nl.estimate()

In [ ]:
results_nl.getEstimatedParameters()

,Value,Active bound,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
B_age_nl,-4.471879e-02,0.0,0.018095,-2.471383e+00,1.345915e-02,0.019448,-2.299397e+00,2.148241e-02
B_age_sq_nl,1.772423e+00,0.0,0.492776,3.596813e+00,3.221397e-04,0.549745,3.224084e+00,1.263761e-03
B_children_nl,1.143103e-01,0.0,0.301266,3.794327e-01,7.043666e-01,0.335084,3.411396e-01,7.329985e-01
B_education_nl,1.765065e-02,0.0,0.044514,3.965178e-01,6.917231e-01,0.045817,3.852402e-01,7.000594e-01
B_gender_nl,-2.999988e-01,0.0,0.217446,-1.379647e+00,1.676953e-01,0.219520,-1.366611e+00,1.717473e-01
B_rating_nl,-5.097309e-01,0.0,0.087897,-5.799208e+00,6.662870e-09,0.091254,-5.585865e+00,2.325393e-08
B_religiousness_nl,-3.588692e-01,0.0,0.088319,-4.063308e+00,4.838203e-05,0.092171,-3.893504e+00,9.880661e-05
B_yearsmarried_nl,2.566418e-01,0.0,0.100844,2.544933e+00,1.092988e-02,0.108225,2.371371e+00,1.772222e-02
B_yearsmarried_sq_nl,-7.792701e-03,0.0,0.005464,-1.426210e+00,1.538076e-01,0.005882,-1.324777e+00,1.852451e-01
delta2,3.757289e-01,0.0,0.062577,6.004240e+00,1.922304e-09,0.060752,6.184640e+00,6.224423e-10



# Exercise 2) Compare the predictions and accuracy of the fine tuned model to a multinomial logit

In this exercise, we have to estimate a standard multinomial model to the affairs dataset, with the alternatives being each of the possible answers of to the survey. **In the multinomial logit, we are dismissing the information about the order relationship between alternatives, but it can still work well in practice.** We will load the auxiliary functions from previous lectures to speed up the process.

In [ ]:
def qbus_estimate_bgm(V, pd_df, tgtvar_name, modelname='bgmdef'):
 av_auto = V.copy()
 for key, value in av_auto.items():
   av_auto[key] = 1
 bgm_db = db.Database(modelname + '_db', pd_df)
 globals().update(bgm_db.variables)
 logprob = models.loglogit (V , av_auto , bgm_db.variables[tgtvar_name] )
 bgm_model = bio.BIOGEME ( bgm_db, logprob )
 bgm_model.utility_dic = V.copy()
 return bgm_model, bgm_model.estimate()

In [ ]:
def qbus_simulate_bgm(qbus_bgm_model, betas, pred_pd_df):
  av_auto = qbus_bgm_model.utility_dic.copy()
  for key, value in av_auto.items():
   av_auto[key] = 1

  targets = qbus_bgm_model.utility_dic.copy()
  for key, value in targets.items():
   targets[key] = models.logit(qbus_bgm_model.utility_dic, av_auto, key)

  bgm_db = db.Database('simul', pred_pd_df)
  globals().update(bgm_db.variables)
  bgm_pred_model = bio.BIOGEME(bgm_db, targets)
  simulatedValues = bgm_pred_model.simulate(betas)
  return simulatedValues

In [ ]:
def qbus_calc_accu_confusion(sim_probs, pd_df, choice_var):
  which_max = sim_probs.idxmax(axis=1)
  data = {'y_Actual':   pd_df[choice_var],
          'y_Predicted': which_max
        }

  df = pd.DataFrame(data, columns=['y_Actual','y_Predicted'])
  confusion_matrix = pd.crosstab(df['y_Actual'], df['y_Predicted'], rownames=['Actual'], colnames=['Predicted'])
  accu = np.mean(which_max == pd_df[choice_var])
  return accu, confusion_matrix 

---

The next step is to define the utility functions for each alternative. Looking at the dataset, we know that the variables are all socioeconomic characteristics, so in order to get different utilities for each of the alternatives, we need to declare per-alternative parameters for the Betas.

Because we have six possible alternatives, we will declare the utility functions using all variables in the dataset **in an automatic way**, we will add betas **for all variables in the input dataset** so we do not have to specify the manually.

Because the occupation variable is not properly addressed in our model (we would need to turn it into dummy indicator variables), we will start by removing it from the dataset.


In [ ]:
affairs_nl = affairs_nl.drop('occupation', axis=1)

The following code cell declares the dictionary with the utility functions that we declared manually in previous tutorials. It does something quite simple, it loops for all possible alternatives in the dataset and creates a utility function that is a linear combination of all variables in the dataset, plus an alternative specific constant. Each alternative is identifies by a suffix '_' and its code number in the dataset.

In [ ]:
V_auto = {}
ref_choiceid = int(affairs_nl['affairs'].unique()[0])
for choiceid in affairs_nl['affairs'].unique():
 choiceid = int(choiceid)
 V_auto[choiceid] = exp.Beta('ASC_' + str(choiceid), 0, None, None, int(choiceid == ref_choiceid))
 for column in affairs_nl.drop('affairs', axis=1):
  V_auto[choiceid] = V_auto[choiceid] + eval(column)*exp.Beta('B_'+ column + '_' + str(choiceid), 0, None, None, 0)

We check the final dictionary to see that it resembles something that we could have done manually.

In [ ]:
V_auto

{0: (((((((((ASC_0(0) + (gender * B_gender_0(0))) + (age * B_age_0(0))) + (yearsmarried * B_yearsmarried_0(0))) + (children * B_children_0(0))) + (religiousness * B_religiousness_0(0))) + (education * B_education_0(0))) + (rating * B_rating_0(0))) + (yearsmarried_sq * B_yearsmarried_sq_0(0))) + (age_sq * B_age_sq_0(0))),
 1: (((((((((ASC_1(0) + (gender * B_gender_1(0))) + (age * B_age_1(0))) + (yearsmarried * B_yearsmarried_1(0))) + (children * B_children_1(0))) + (religiousness * B_religiousness_1(0))) + (education * B_education_1(0))) + (rating * B_rating_1(0))) + (yearsmarried_sq * B_yearsmarried_sq_1(0))) + (age_sq * B_age_sq_1(0))),
 2: (((((((((ASC_2(0) + (gender * B_gender_2(0))) + (age * B_age_2(0))) + (yearsmarried * B_yearsmarried_2(0))) + (children * B_children_2(0))) + (religiousness * B_religiousness_2(0))) + (education * B_education_2(0))) + (rating * B_rating_2(0))) + (yearsmarried_sq * B_yearsmarried_sq_2(0))) + (age_sq * B_age_sq_2(0))),
 3: (((((((((ASC_3(0) + (gender

We can estiamte the model using the auxiliary function as we have done in the past.



In [ ]:
MNL_model, MNL_results = qbus_estimate_bgm(V_auto, affairs_nl, 'affairs')

There are too many coefficients, 59!. So we will check a few of them for correctness.

In [ ]:
MNL_results.getEstimatedParameters().head(15)

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_1,-0.027475,2.917095,-0.009419,0.992485,2.486378,-0.011050,0.991183
ASC_12,1.969813,3.504788,0.562035,0.574092,4.271549,0.461147,0.644693
ASC_2,3.113478,4.434848,0.702048,0.482649,3.842087,0.810361,0.417733
ASC_3,-0.950718,4.449179,-0.213684,0.830794,3.705425,-0.256575,0.797507
ASC_7,-5.165264,3.283770,-1.572968,0.115726,3.318233,-1.556631,0.119558
B_age_0,0.072514,0.088082,0.823255,0.410363,0.083327,0.870234,0.384173
B_age_1,0.037219,0.153060,0.243164,0.807878,0.147633,0.252103,0.800961
B_age_12,0.071896,0.174904,0.411058,0.681030,0.178747,0.402220,0.687522
B_age_2,-0.485818,0.202824,-2.395271,0.016608,0.203162,-2.391280,0.016790
B_age_3,0.222788,0.221685,1.004976,0.314908,0.129184,1.724584,0.084602


Finally we simulate the predictions and check the accuracy, we get a 75.6% accuracy.

In [ ]:
MNL_preds = qbus_simulate_bgm(MNL_model, MNL_results.getBetaValues(), affairs_nl)
qbus_calc_accu_confusion(MNL_preds, affairs_nl, 'affairs')[0]

0.7554076539101497

The ordered logit, on the other hand, will have an accuracy of 74.9%, but it has way fewer parameters.

In [ ]:
  bgm_pred_model = bio.BIOGEME(affairs_nl_db, alt_probs_map_nl)
  simulatedValues_nl = bgm_pred_model.simulate(results_nl.getBetaValues())

In [ ]:
qbus_calc_accu_confusion(simulatedValues_nl, affairs_nl, 'affairs')[0]

0.7487520798668885

When we check the naive predictions, just the proportion of the dataset, we get that the models do not predict much better in terms of accuracy, since the baseline accuracy for a trivial model will already get 75% accuracy.

In [ ]:
pd.crosstab(affairs_pd['affairs'], 'prop') / affairs_pd.shape[0]

col_0,prop
affairs,
0,0.750416
1,0.056572
2,0.028286
3,0.031614
7,0.069884
12,0.063228


But even when the accuracies are not much better, maybe the difference might lie in the actual choice probabilities, so we can check them for the MNL first, 
and the for the ordered logit. The diference does not seem that great, so we can conclude that either both models are poor, or they are both OK, while the ordered logit required way less parameters, it is much more interpretable.

In [ ]:
MNL_preds.mean(axis=0).sort_index()

0     0.750416
1     0.056573
2     0.028287
3     0.031613
7     0.069883
12    0.063228
dtype: float64

In [ ]:
simulatedValues_nl.mean(axis=0)

0     0.745001
1     0.057140
2     0.029285
3     0.032429
7     0.071985
12    0.064161
dtype: float64